<a href="https://colab.research.google.com/github/MarioROT/IHLT-MAI/blob/main/Session3_MarioRosas_AlamLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
```

# Lab session 3 (Morphology) - IHLT

**Students:**
- Mario Rosas
- Alam Lopez

**Lab Professor:** Salvador Medina Herrera

## Paraphrases Template

In [3]:
%%shell
git clone https://github.com/mariorot/IHLT-MAI.git
cd 'IHLT-MAI'
mv 'complementary_material' /content/
mv scripts /content/

Cloning into 'IHLT-MAI'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 117 (delta 60), reused 56 (delta 19), pack-reused 0
Receiving objects: 100% (117/117), 159.00 KiB | 2.01 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [1]:
from scripts.compute_metrics import ComputeMetrics
from scripts.text_preprocessing import TextPreprocessing
import pandas as pd
from scipy.stats import pearsonr


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
dt = pd.read_csv('complementary_material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)
dt['gs'] = pd.read_csv('complementary_material/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

## TODO
1. Read all pairs of sentences of the SMTeuroparl files of test set within the evaluation framework of the project. Compute their similarities by considering *lemmas* and *Jaccard distance*. <br>

2. Compare the results with those in session 2 (document structure) in which words were considered. : <br>

3. Compare the results with gold standard by giving the pearson correlation between them. <br>

4. Questions (justify the answers): <br>

      Which is better: words or lemmas?

      Do you think that could perform better for any pair of texts?
```
from scipy.stats import pearsonr
pearsonr(dt['gs'], dt['jaccard'])[0]
```

In [3]:
 tp = TextPreprocessing()

 dt[2]=tp.clean_data(list(dt[0]))
 dt[3]=tp.clean_data(list(dt[1]))

The code for the tokenization is at: https://github.com/MarioROT/IHLT-MAI/blob/main/scripts/text_preprocessing.py

The code for jaccard distance computation is at: https://github.com/MarioROT/IHLT-MAI/blob/main/scripts/compute_metrics.py

In [4]:

dt[4] = tp.tokenize_data(list(dt[0]),'nltk')
dt[5] = tp.tokenize_data(list(dt[1]),'nltk')
cm_nltk = ComputeMetrics(dt[[4,5]].to_numpy(), ['jaccard'], 1)
dt['jaccard_token_nltk'] = cm_nltk.do()['jaccard']

cm_clean_nltk = ComputeMetrics(dt[[2,3]].to_numpy(), ['jaccard'], 1)
dt['jaccard_token_clean_nltk'] = cm_clean_nltk.do()['jaccard']


dt[8] = tp.tokenize_data(list(dt[0]),'spacy')
dt[9] = tp.tokenize_data(list(dt[1]),'spacy')
cm_spacy = ComputeMetrics(dt[[8,9]].to_numpy(), ['jaccard'], 1)
dt['jaccard_token_spacy'] = cm_spacy.do()['jaccard']


cm_clean_spacy = ComputeMetrics(dt[[2,3]].to_numpy(), ['jaccard'], 1)
dt['jaccard_token_clean_spacy'] = cm_clean_spacy.do()['jaccard']



Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...


In [8]:
dt[12]=tp.lemmatize_data(list(dt[0]),'spacy')
dt[13]=tp.lemmatize_data(list(dt[1]),'spacy')
cm_lemma_spacy = ComputeMetrics(dt[[12,13]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_spacy']=cm_lemma_spacy.do()['jaccard']

dt[14]=tp.lemmatize_data(list(dt[2]),'spacy')
dt[15]=tp.lemmatize_data(list(dt[3]),'spacy')
cm_lemma_clean_spacy = ComputeMetrics(dt[[14,15]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_clean_spacy']=cm_lemma_clean_spacy.do()['jaccard']

dt[16]=tp.lemmatize_data(list(dt[0]),'nltk',False)
dt[17]=tp.lemmatize_data(list(dt[1]),'nltk',False)
cm_lemma_nltk = ComputeMetrics(dt[[16,17]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_nltk']=cm_lemma_nltk.do()['jaccard']

dt[18]=tp.lemmatize_data(list(dt[2]),'nltk')
dt[19]=tp.lemmatize_data(list(dt[3]),'nltk')
cm_lemma_clean_nltk = ComputeMetrics(dt[[18,19]].to_numpy(), ['jaccard'], 1)
dt['jaccard_lemma_clean_nltk']=cm_lemma_clean_nltk.do()['jaccard']


Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...


In [6]:
dt[['gs','jaccard_token_nltk','jaccard_token_clean_nltk','jaccard_token_spacy','jaccard_token_clean_spacy','jaccard_lemma_nltk','jaccard_lemma_clean_nltk','jaccard_lemma_spacy','jaccard_lemma_clean_spacy']]

,gs,jaccard_token_nltk,jaccard_token_clean_nltk,jaccard_token_spacy,jaccard_token_clean_spacy,jaccard_lemma_nltk,jaccard_lemma_clean_nltk,jaccard_lemma_spacy,jaccard_lemma_clean_spacy
0,4.500,0.653846,0.692308,0.653846,0.692308,0.653846,0.666667,0.600000,0.666667
1,5.000,0.214286,0.285714,0.214286,0.285714,0.076923,0.000000,0.076923,0.000000
2,4.250,0.608696,0.727273,0.608696,0.727273,0.608696,0.727273,0.545455,0.727273
3,4.500,0.454545,0.250000,0.400000,0.250000,0.454545,0.250000,0.400000,0.250000
4,5.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
454,5.000,0.450000,0.636364,0.450000,0.636364,0.450000,0.636364,0.450000,0.636364
455,4.750,0.642857,0.250000,0.714286,0.250000,0.642857,0.250000,0.615385,0.250000
456,5.000,0.600000,0.666667,0.600000,0.666667,0.600000,0.666667,0.526316,0.666667
457,4.000,0.681818,0.800000,0.681818,0.800000,0.681818,0.800000,0.619048,0.800000


In [7]:
p_token_nltk=pearsonr(dt['gs'], 1-dt['jaccard_token_nltk'])[0] # Calculating the pearson correlation between GS results and 1-Jaccard calculated dats
p_token_nltk_clean=pearsonr(dt['gs'], 1-dt['jaccard_token_clean_nltk'])[0]
p_token_spacy=pearsonr(dt['gs'], 1-dt['jaccard_token_spacy'])[0]
p_token_spacy_clean=pearsonr(dt['gs'], 1-dt['jaccard_token_clean_spacy'])[0]
p_lemma_nltk=pearsonr(dt['gs'], 1-dt['jaccard_lemma_nltk'])[0] # Calculating the pearson correlation between GS results and 1-Jaccard calculated dats
p_lemma_nltk_clean=pearsonr(dt['gs'], 1-dt['jaccard_lemma_clean_nltk'])[0]
p_lemma_spacy=pearsonr(dt['gs'], 1-dt['jaccard_lemma_spacy'])[0]
p_lemma_spacy_clean=pearsonr(dt['gs'], 1-dt['jaccard_lemma_clean_spacy'])[0]


print(f'Token:  NLTK {p_token_nltk} --- NLTK clean {p_token_nltk_clean} --- spaCy {p_token_spacy} --- spaCy clean {p_token_spacy_clean}')
print(f'Lemma:  NLTK {p_lemma_nltk} --- NLTK clean {p_lemma_nltk_clean} --- spaCy {p_lemma_spacy} --- spaCy clean {p_lemma_spacy_clean}')


Token:  NLTK 0.45049771693186846 --- NLTK clean 0.4679440754704451 --- spaCy 0.4608504322255335 --- spaCy clean 0.4679440754704451
Lemma:  NLTK 0.4492874506144061 --- NLTK clean 0.4810231734170825 --- spaCy 0.4767768522225688 --- spaCy clean 0.49124538340870183


# Conclusion

Which is better: words or lemmas?

Do you think that could perform better for any pair of texts?


The use of different tokenizers (NLTK and spaCy) can impact the results when comparing sentence pairs, as they may perform differently depending on the input text data. Jaccard similarity proved to be an effective metric for comparing the similarity of sentence pairs after word tokenization. With the inversion of this metric we got a distance to measure the dissimilarity between sets of tokens, making it suitable for the task we were trying to solve. The Pearson correlation coefficient provided us a quantitative measure of how well our results align with the gold-standard, i.e. effectiveness.

The results indicate that spaCy slightly outperformed NLTK in the task of sentence pair comparison using Jaccard distance.The Pearson correlation coefficient values obtained for **NLTK (0.4505)** and **spaCy (0.4609)** suggest a moderate positive correlation between the computed distances and the benchmark (gold-standard) distances.

Our best result could be possibly improved by experimenting with different tokenization strategies or exploring other NLP techniques to enhance the performance.


